## Vector-valued function with Random Fourier Features

### Imports

In [133]:
import numpy as np
from numpy.random import randn

### Values

In [134]:

dimension = 2
sigma = 2
sigma2 = 2*sigma**2

# Random Fourier features
m = 5 # number of samples
w = randn(dimension*m) / sigma # w is the same size as the dimensions times the number of samples
b = np.random.rand(m)*2*np.pi # b is the same size as number of samples

n = 100 # number of points

x1 = randn(n)/2
x2 =  randn(n)/2
x_vector = np.array([x1, x2])
# x_vector = np.block([x1,x2])

noise = randn(n) * 2e-1
y1 = np.cos(x1) + randn(n) * 1e-1
y2 = np.sin(x2) + randn(n) * 1e-1


#y1 = 1+np.sin(x1/10) + x1**2
#y2 = 0.5 * x2**2 + np.exp(x2)

y = np.block([y1, y2])

#print(y1, y2)
lam = 0.000001 # Regularisaton parameter

### Gaussian Kernel

In [135]:
def k_gauss(x, z):
    k = np.exp(-((np.linalg.norm(x-z))**2)/(sigma2))
    return k

def K_gauss(x, n, dim):
    K = np.zeros((dim*n, dim*n))
    for i in range(0, n):
        for j in range(0, n):
            k = k_gauss(x[:,i], x[:,j])
            K[i*dim][j*dim] = k
            K[i*dim+1][j*dim+1] = k
    return K

K = K_gauss(x_vector, n, dimension)
print(K)

[[1.         0.         0.99729643 ... 0.         0.90778074 0.        ]
 [0.         1.         0.         ... 0.94205867 0.         0.90778074]
 [0.99729643 0.         1.         ... 0.         0.89186838 0.        ]
 ...
 [0.         0.94205867 0.         ... 1.         0.         0.96894598]
 [0.90778074 0.         0.89186838 ... 0.         1.         0.        ]
 [0.         0.90778074 0.         ... 0.96894598 0.         1.        ]]


#### Alpha

In [136]:
# Returns a vector with length dim * n
def alpha(x, y, lam, n, dim):
    K = K_gauss(x, n, dim)
    alpha_vector = np.linalg.inv((K + lam*np.eye(dim*n))) @ y
    alpha = np.reshape(alpha_vector, (dim, n))
    return alpha

alpha_ = alpha(x_vector, y, lam, n, dimension)


[[  540775.91145858   560825.87390682   344852.68434652   737673.34501493
    481646.66509053   306559.89900391   559735.94747655   794332.70614082
      5399.52292193   -79932.81235049   400232.905838     198143.30889641
    720394.08189319  1071069.420943     586047.91335157   532979.95621621
   -157367.18471076   -24468.28992494   156741.50841159   448122.72812676
    492153.77813272   525956.97346341   108138.89115038   290133.53472363
    447837.31385363   407572.11870993   529492.92359996   544454.2267577
     -9650.13855629   162943.42938043   302936.64298713   355894.13380489
    502430.36972214   690305.84835793    -3873.67689023    83554.2907443
     66130.8540311    -67271.82464703   568534.84859425   507100.73414273
    521038.69564407   305748.33507104   625393.96545471   595671.48457948
    324958.60952635   279224.58048649   263051.69891186    62767.19735681
     45970.14100204   681245.98648473   236030.13504868   185059.71316193
    300059.22000356   418199.13006908   

### Estimated function

In [140]:
def estiamted_function(alpha, x, n):
    function = np.zeros((2, n))
    for i in range(n):
        sum1 = 0
        #sum2 = 0
        for j in range(n):
            k = k_gauss(x[0, i], x[0, j])
            sum1 = sum1 + k * alpha[0, j]
            #sum2 = sum2 + k * alpha[1, j]
        function[0, i] = sum1
        #function[1, i] = sum2
    return function

estiamted_function(alpha_, x_vector, n)

array([[34653105.73042433, 34506393.98015544, 33426065.86800936,
        34824844.41461597, 34638587.71463183, 32129821.83080854,
        34557713.81180982, 35022526.47760731, 31709628.63838986,
        30221979.75104962, 35006004.15508771, 31228861.96518304,
        34964535.18529719, 35012454.79167166, 34561121.09365881,
        34823637.84373942, 34548383.62991732, 29179817.83578746,
        28232625.15658292, 34899651.85441986, 34776383.16605752,
        35026797.33463233, 34953077.22854351, 30731393.18693661,
        33433612.04011055, 34494397.71619626, 34744628.43035907,
        35025798.87375247, 34111187.22875938, 31315131.60318263,
        34313459.70515127, 34744465.82882621, 35025279.99571869,
        35017904.39926155, 31655998.6396636 , 32067570.43278799,
        34973499.74072428, 35000659.46181806, 34172616.83848526,
        34897721.90310927, 34394788.11231085, 33603727.39886563,
        34961762.42602915, 35021395.91226986, 33818044.97255045,
        33139603.03089586